# ONLY RUN THE NEXT THREE CELL's ONCE!

## Change the runtime Edit --> Notebook Settings --> Runtime Version --> 2025.07

# MC things to fix!
- File names, and paths to be consistent
- Add more comments and explanations

In [ ]:
# Global variable to check if we're in Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    ! pip install numpy==1.26.4
    ### The runtime will crash/restart after this

In [ ]:
# Global variable to check if we're in Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:

  !pip install -q condacolab #-U "git+https://github.com/mayankchetan/condacolab.git@py312update"
  import condacolab
  condacolab.install() #### expect a kernel restart

In [ ]:
# Global variable to check if we're in Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !git clone  https://github.com/wisdem/weis
    !git clone https://github.com/mayankchetan/weisWorkshop

    %cd weis
    !conda config --env --add pinned_packages "numpy<2.0"
    !mamba env update -n base -f environment.yml "python<3.13"
    %pip install .
    %cd ..
    %pip install "rosco==2.9.5"
    %pip install -U "git+https://github.com/NREL/ROSCO@v2.9.7"
    %pip install "orbit-nrel<1.2.2"
    %pip install gdown

else:
    try:
        import weis

    except ImportError:
        raise ImportError("Please install the weis package from https://github.com/wisdem/weis in the environment you are using to run this notebook.")

    Warning("Make sure you have cloned 'https://github.com/mayankchetan/weisWorkshop-Private' repository and are running the notebook from the root directory of the repository.")


In [ ]:
# Downloading the precomputed data from Google Drive & extracting
# Drive link for non Google Colab users: https://drive.google.com/file/d/145BH393nzpT2msCVsHSBxR3Qy8KdpddI/view?usp=sharing
if IN_COLAB:
    %gdown --id 145BH393nzpT2msCVsHSBxR3Qy8KdpddI -O /content/weisWorkshop/archive_weisWorkshop_OCT25.tar.gz
    %tar -xvzf /content/weisWorkshop/archive_weisWorkshop_OCT25.tar.gz -C /content/weisWorkshop/
else:
    print("Please download the precomputed data from the link given in the notebook and extract it in the root directory of the repository.")
    print("you can use `tar -xvzf <downloaded_file_name>` to extract the files.")

# Welcome to session 2 of the WEIS workshop!
 > Write some info

# Recap
- In the last session, we ran the IEA 22MW floating wind turbine in WEIS, and established a baseline
- We modified the rotor diameter and rated power to create a 20MW turbine
- We optimized the rotor using WISDEM
- We inspected the optimization log using the log_opt.sql file, and plotted the convergence

# The next logical steps
- Now that we have a new rotor, we need to re-tune the controller so that we are not operating in an inefficient manner, and the loads on the system are better managed.
- We will use the ROSCO within WEIS to re-tune the controller for the new rotor.
- Using this new rotor and controller, we will begin to optimize the floating substructure to further reduce cost
- Finally, we will compare the baseline IEA 22MW turbine with the fully optimized 20MW turbine
- We'll also setup a subset of Design Load Cases (DLCs) to inspect the loads on the system

In [ ]:
if IN_COLAB:
    # Now we try loading the weis_workshop module again
    sys.path.append('/content/weisWorkshop-Private/jupyterNotebooks')

# Importing nesscary libraries.
import numpy as np
import matplotlib.pyplot as plt
from weis import weis_main
from wisdem import run_wisdem
import os
import copy
import pandas as pd

from weis_workshop import *

# set env variables
os.environ["OMP_NUM_THREADS"] = "1"  # export OMP_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1"  # export MKL_NUM_THREADS=1

# workshop repository folder
if IN_COLAB:
    WORKSHOP_REPO = '/content/weisWorkshop-Private'
else:
    WORKSHOP_REPO = os.path.join(os.getcwd(), '..')

# Now Focus on content and will make additional text, notes later

## Stage 3: Controller Tuning

In [ ]:
# Setting up file paths
stageFolder = "stage-2-controller"

fname_wt_input = os.path.join(WORKSHOP_REPO, "stage-1-aeroStruct","outputs_preCompute", "stage-1-aeroStruct-aero_analysis.yaml")
fname_modeling_options = os.path.join(WORKSHOP_REPO, stageFolder, f"stage-2-controller_modeling.yaml")
fname_analysis_options = os.path.join(WORKSHOP_REPO, stageFolder, f"stage-2-controller_analysis.yaml")


# Setting up the Optmization problem

<write some info>


## Analysis Options (just the important bits):
```yaml

design_variables:
  control:
    servo:
      pitch_control:
          omega:
            flag: True
            min: 0.025
            max: 0.5
          zeta:
            flag: True
            min: 0.1
            max: 3.0
          Kp_float:
            flag: True
            min: -40.0  # -0.25
            max: 0
          ptfm_freq:
            flag: True
            max: 0.5

merit_figure: DEL_TwrBsMyt  # Merit figure of the optimization problem. The options are 'AEP' - 'LCOE' - 'Cp' - 'blade_mass' - 'blade_tip_deflection'

constraints:
  control:
    rotor_overspeed:
      flag: True
      min: 0.0
      max: 0.2

driver:
  optimization:
    flag: True
    tol: 1.e-3            # Optimality tolerance
    max_iter: 200         # Maximum number of iterations (SLSQP)
    maxiter: 60          # Maximum number of iterations (SLSQP)
    solver: LN_COBYLA         # Optimization solver. Other options are 'SLSQP' - 'CONMIN'
    step_size: 1.e-3      # Step size for finite differencing
    form: forward         # Finite differencing mode, either forward or central
```

## Modeling Options (just the important bits):
``` yaml
General:
    verbosity: False  # When set to True, the code prints to screen many infos
    openfast_configuration:
        OF_run_fst: stage-2-controller
        save_iterations: True
        save_timeseries: True
        use_exe: True
        allow_fails: True
        fail_value: 9999
        write_stdout: True

WISDEM:
    RotorSE:
        flag: True
        n_pitch_perf_surfaces: 20
        n_tsr_perf_surfaces: 20
        spar_cap_ss: spar_cap_ss
        spar_cap_ps: spar_cap_ps
        peak_thrust_shaving: True
        thrust_shaving_coeff: 0.8
    TowerSE:
        flag: True
        wind: PowerWind  # Wind used
        gamma_f: 1.35    # Safety factor for fatigue loads
        gamma_m: 1.3     # Safety factor for material properties
        gamma_n: 1.0     # Safety factor for ...
        gamma_b: 1.1     # Safety factor for ...
        gamma_fatigue: 1.755  # Safety factor for fatigue loads
        buckling_method: dnvgl # Buckling code type [eurocode or dnvgl]
        buckling_length: 15    # Buckling parameter
        frame3dd:
            shear: True
            geom: True
            tol: 1e-9
    DriveSE:
        flag: True
    FloatingSE:
        flag: True
        symmetric_moorings: True
        gamma_f: 1.35    # Safety factor for fatigue loads
        gamma_m: 1.3     # Safety factor for material properties
        gamma_n: 1.0     # Safety factor for ...
        gamma_b: 1.1     # Safety factor for ...
        gamma_fatigue: 1.755  # Safety factor for fatigue loads
        rank_and_file: True
    BOS:
        flag: True

OpenFAST: # Options for WEIS fidelity level 3 = nonlinear time domain
    flag: True
    simulation:
        DT: 0.01
        CompElast: 1
        CompInflow: 1
        CompAero: 2
        CompServo: 1
        CompHydro: 1
        CompSub: 0
        CompMooring: 3
        CompIce: 0
        OutFileFmt: 3
        NumCrctn: 1
    ElastoDyn:
        FlapDOF1: True
        FlapDOF2: True
        EdgeDOF: True
        TeetDOF: False
        DrTrDOF: False
        GenDOF: True
        YawDOF: False
        TwFADOF1 : True
        TwFADOF2 : True
        TwSSDOF1 : True
        TwSSDOF2 : True
        PtfmSgDOF: True
        PtfmSwDOF: True
        PtfmHvDOF: True
        PtfmRDOF : True
        PtfmPDOF : True
        PtfmYDOF : True
RAFT:
    flag: True
    potential_model_override: 0
    trim_ballast: 2
    heave_tol: 1
    save_designs: True
ROSCO:
    flag: True
    tuning_yaml: ../source/iea22_rosco.yaml

DLC_driver:
    metocean_conditions:
        wind_speed: [4., 6., 8., 10., 12., 14., 16., 18., 20., 22., 24.]
        wave_height_NSS: [0.83, 0.88, 0.94, 1.03, 1.16, 1.34, 1.57, 1.86, 2.22, 2.62, 3.07]
        wave_period_NSS: [6.9, 6.96, 7.02, 7.12, 7.25, 7.43, 7.66, 7.94, 8.27, 8.63, 9.01]
        wave_height_SSS: [6.3, 8, 8, 8.1, 8.5, 8.5, 9.8, 9.8, 9.8, 9.8, 9.9]
        wave_period_SSS: [11.5, 12.7, 12.7, 12.8, 13.1, 13.1, 14.1, 14.1, 14.1, 14.1, 14.1]
        wave_height_1: 6.98
        wave_period_1: 11.7
        wave_height_50: 10.68
        wave_period_50: 14.2
    DLCs:
        - DLC: "1.1"
          n_seeds: 6
          transient_time: 120.0  # 0.
          analysis_time: 600.0  # 10.
```

In [ ]:
# lets overide a few things so that we dont distrupt the existing results.
analysis_override = {}
analysis_override['general'] = {}
analysis_override['general']['folder_output'] = 'outputsCh1'
analysis_override['driver'] = {}
analysis_override['driver']['optimization'] = {}
analysis_override['driver']['optimization']['max_iter'] = 1 # only run one iteration for this demo, we have precomputed the solutions for you :)


modeling_override = {}
modeling_override['DLC_driver'] = {}
modeling_override['DLC_driver']['DLCs'] = [
    {'DLC': "1.1",
     'n_seeds': 1,
     'wind_speed': [20.0],
     'transient_time': 10.0,
     'analysis_time': 30.0}]

control_20mw, modeling_options, opt_options = weis_main(fname_wt_input, 
                                                 fname_modeling_options, 
                                                 fname_analysis_options,
                                                 analysis_override=analysis_override,
                                                 modeling_override=modeling_override,
                                                 test_run=False
                                                 )

We will do the analysis of the controller tuning using the pre-computed results

In [ ]:
rec_data = load_OMsql(os.path.join(WORKSHOP_REPO, stageFolder,"outputs_preCompute/log_opt.sql")) # not sure what supresses the output here
plot_convergence(rec_data, ['aeroelastic.DEL_TwrBsMyt', 'aeroelastic.rotor_overspeed', 'sse_tune.tune_rosco.PC_Ki', 
                            'sse_tune.tune_rosco.PC_Kp', 'tune_rosco_ivc.Kp_float', 'tune_rosco_ivc.omega_pc', 
                            'tune_rosco_ivc.ptfm_freq', 'tune_rosco_ivc.zeta_pc'])

# Stage 3: Floating Substructure Optimization
- Now that we have a new rotor, we need to re-tune the controller so that we are not operating in an inefficient manner, and the loads on the system are better managed.
- We will follow Zalkind et al (2024) "Control Co-Design Studies for a 22 MW Semisubmersible Floating Wind Turbine Platform" DOI 10.1088/1742-6596/2767/8/082020 to set up the optimization problem
    - So the first step will be to use RAFT to size the floating substructure
    - Then we use the results from RAFT to set up the optimization problem using OpenFAST for higher fidelity analysis
    - We will use the pre-computed results for this part as well
    - We have also pre-computed a floater + controller Co-Design case

In [ ]:
# Setting up file paths
stageFolder = "stage-3-semisub" # First the RAFT optimization

fname_wt_input = os.path.join(WORKSHOP_REPO, "stage-2-controller","outputs_preCompute", "stage-2-controller.yaml")
fname_modeling_options = os.path.join(WORKSHOP_REPO, stageFolder, f"stage-3-semisub_raft_modeling.yaml")
fname_analysis_options = os.path.join(WORKSHOP_REPO, stageFolder, f"stage-3-semisub_raft_analysis.yaml")

# Talk about input 


# We wont overide the modeling options since RAFT has lower computational cost, but limit it to a single iteration.

In [ ]:
# lets overide a few things so that we dont distrupt the existing results.
analysis_override = {}
analysis_override['general'] = {}
analysis_override['general']['folder_output'] = 'outputsCh1'
analysis_override['driver'] = {}
analysis_override['driver']['optimization'] = {}
analysis_override['driver']['optimization']['max_iter'] = 1 # only run one iteration for this demo, we have precomputed the solutions for you :)


control_20mw, modeling_options, opt_options = weis_main(fname_wt_input, 
                                                 fname_modeling_options, 
                                                 fname_analysis_options,
                                                 analysis_override=analysis_override,
                                                #  modeling_override=modeling_override,
                                                 test_run=False
                                                 )

In [ ]:
rec_data = load_OMsql(os.path.join(WORKSHOP_REPO, stageFolder,"outputs_preCompute/log_opt.sql")) # not sure what supresses the output here
plot_convergence(rec_data, ['raft.Max_PtfmPitch', 'raft.heave_period', 'raft.max_nac_accel', 'raft.pitch_period', 
                            'floatingse.constr_draft_heel_margin', 'floatingse.constr_fairlead_wave', 'floatingse.constr_fixed_margin', 
                            'floatingse.constr_freeboard_heel_margin', 'floatingse.metacentric_height_pitch', 'floatingse.metacentric_height_roll', 
                            'floatingse.constr_variable_margin', 'floatingse.system_structural_mass', 'floating.jointdv_0', 
                            'floating.jointdv_1', 'floating.memgrp1.outer_diameter_in'])



